In [124]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
from datetime import date
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

from config import NEWS_API_KEY
from api_helpers import *
from sim import *

In [192]:
data = news_top()

In [196]:
table_data = []
for article in data['articles']:
    print("----------------")
    query = article['title']
    if article['description'] is not None:
        query += article['description']
        
    query = query.replace(article['source']['name'], '') # remove publication information
    
    print(query)
    
    # start to break query into named entities
    doc = nlp(query)

    entities = []
    for ent in doc.ents:
        if (ent.label_ in ['PERSON', 'FAC', 'ORG']) and (ent.text not in entities):
            entities.append(ent.text)

    # search wikipedia for each entry
    print(entities)
    entities_d = {}
    titles = []
    revisions = []
    for entity in entities:
        articledata = wiki_query(entity) # wiki
        
        try: 
            search_results= articledata['query']['search']
            i = 0
            while((i < len(search_results)) & (distance(entity, search_results[i]['title']) < 10)):
                title = search_results[i]['title']
                info = wiki_pageinfo(title)
                info = list(info['query']['pages'].values())[0]
                
                
                categories = [kv['title'] for kv in info['categories'] if 'births' not in kv['title']]
                # ignore diambiguation pages
                if 'Category:Disambiguation pages' in categories:
                    pass
                else:
                    content = list(wiki_pagecontent(title)['query']['pages'].values())[0]
                    if 'thumbnail' in content:
                        image = content['thumbnail']['source']
                    else:
                        image = None
                    table_data.append({'news': query,
                                       'entity': entity,
                                       'title': info['title'],
                                       'revised': info['touched'],
                                       'url': info['fullurl'],
                                       'wiki': content['extract'],
                                       'image': image})
                i += 1
        
        except(IndexError):
            continue

----------------
L.A. County set for major reopening as it hits yellow tier - Moving into the yellow tier clears the way for Los Angeles County to unshackle its economy to the widest extent currently possible.
[]
----------------
Terry Bradshaw calls Aaron Rodgers 'weak' over Packers drama, says he should 'go ahead and retire' - Aaron Rodgers’ growing rift with the Green Bay Packers stole the spotlight last week just hours ahead of the 2021 NFL Draft – but Hall of Fame quarterback and FOX Sports broadcaster Terry Bradshaw called Rodgers "weak" for it, adding: "He should just retire."
['Terry Bradshaw', 'Aaron Rodgers', 'NFL', 'Rodgers']
----------------
Pfizer Says FDA Will Soon Authorize COVID-19 Vaccine For 12-15 Age Group - The company said in late March that clinical trials showed the vaccine elicits "100% efficacy and robust antibody responses" in adolescents.
['FDA', 'Age Group']
----------------
Epic pushed Xbox chief to open free multiplayer just ahead of Apple Fortnite battle 

----------------
NOAA unveils new U.S. climate ‘normals’ that are warmer than ever - Drawing from the latest decade of weather data, the new normals are a reflection of climate change.
['NOAA']
----------------
CVS Health earnings top estimates as Covid vaccinations, testing boost drugstore sales - CVS Health reported a strong first quarter and raised its forecast, as customers came to  stores for Covid-19 vaccinations, tests and prescriptions.
['CVS Health']
----------------
After Israel stampede, some ultra-Orthodox are looking at their role in the tragedy - Days after a deadly stampede resulted in the deaths of 45 people at a religious festival in northern Israel, many are now asking who is at fault.
[]
----------------
Why Lagging COVID Vaccine Rate At Rural Hospitals 'Needs To Be Fixed Now' - When it comes to persuading people to get a COVID-19 vaccine, health care professionals are influential. But in some rural communities, hospital staff themselves are hanging back.
["Rural Hos

In [205]:
table = pd.DataFrame(table_data)
table['sim'] = table[['news', 'wiki']].apply(lambda x: sim_sm(x[0], x[1]), axis=1)
table.head()

,news,entity,title,revised,url,wiki,image
0,Terry Bradshaw calls Aaron Rodgers 'weak' over...,Terry Bradshaw,Terry Bradshaw,2021-05-03T05:21:22Z,https://en.wikipedia.org/wiki/Terry_Bradshaw,"Terry Paxton Bradshaw (born September 2, 1948)...",https://upload.wikimedia.org/wikipedia/commons...
1,Terry Bradshaw calls Aaron Rodgers 'weak' over...,Aaron Rodgers,Aaron Rodgers,2021-05-04T15:31:55Z,https://en.wikipedia.org/wiki/Aaron_Rodgers,"Aaron Charles Rodgers (born December 2, 1983) ...",https://upload.wikimedia.org/wikipedia/commons...
2,Terry Bradshaw calls Aaron Rodgers 'weak' over...,Aaron Rodgers,Jordan Rodgers,2021-05-01T07:01:09Z,https://en.wikipedia.org/wiki/Jordan_Rodgers,"Jordan Edward Rodgers (born August 30, 1988) i...",https://upload.wikimedia.org/wikipedia/commons...
3,Terry Bradshaw calls Aaron Rodgers 'weak' over...,Rodgers,Aaron Rodgers,2021-05-04T15:31:55Z,https://en.wikipedia.org/wiki/Aaron_Rodgers,"Aaron Charles Rodgers (born December 2, 1983) ...",https://upload.wikimedia.org/wikipedia/commons...
4,Terry Bradshaw calls Aaron Rodgers 'weak' over...,Rodgers,Brendan Rodgers,2021-04-30T21:14:04Z,https://en.wikipedia.org/wiki/Brendan_Rodgers,Brendan Rodgers (born 26 January 1973) is a No...,https://upload.wikimedia.org/wikipedia/commons...


In [204]:
today = date.today().strftime("%b-%d-%Y")
table.to_csv(f'{today}.csv', index = False)